In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

DATA_PATH = Path(r"Drug_overdose_death_rates__by_drug_type__sex__age__race__and_Hispanic_origin__United_States.csv")
plt.style.use('seaborn-whitegrid')

In [ ]:
df = pd.read_csv(DATA_PATH)
df.columns = [col.strip().lower().replace(' ', '_').replace('/', '_').replace('(', '').replace(')', '') for col in df.columns]
df['estimate'] = pd.to_numeric(df['estimate'], errors='coerce')
df['is_flagged'] = df['flag'].notna().astype(int)
df = df.drop(columns=['panel_num', 'stub_name_num', 'stub_label_num', 'year_num', 'age_num', 'flag', 'unit', 'indicator'])
df = df.dropna(subset=['estimate']).reset_index(drop=True)
print(f'Tamanho da base após limpeza: {len(df)} registros')

## Análise Exploratória e Visão Geral
Veja algumas estatísticas descritivas e como a taxa evolui ao longo dos anos por categoria principal (`panel`).

In [ ]:
display(df.head())
display(df[['year', 'panel', 'stub_label', 'estimate']].describe())

# Distribuição da variável alvo
plt.figure(figsize=(10, 4))
sns.histplot(df['estimate'], bins=30, kde=True)
plt.title('Distribuição das Taxas de Morte por Overdose')
plt.xlabel('Taxa (por 100k hab)')
plt.show()

# Boxplot para identificar outliers por categoria principal
plt.figure(figsize=(12, 6))
sns.boxplot(data=df, x='estimate', y='panel')
plt.title('Boxplot das Taxas por Painel (Identificação de Outliers)')
plt.show()

# Evolução temporal por idade
plt.figure(figsize=(12, 5))
sns.lineplot(data=df[df['stub_name'] == 'Age'], x='year', y='estimate', hue='stub_label')
plt.title('Taxas por faixa etária ao longo dos anos')
plt.ylabel('Morte por 100 mil habitantes')
plt.legend(bbox_to_anchor=(1.01, 1), loc='upper left')
plt.tight_layout()
plt.show()

## Pré-processamento e engenharia de atributos
Vamos transformar variáveis categóricas com one-hot encoding e padronizar numéricas para alimentar modelos clássicos de regressão.

In [ ]:
categorical_cols = ['panel', 'stub_name', 'stub_label', 'age']
numeric_cols = ['year', 'is_flagged']
preprocessor = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols),
    ('num', StandardScaler(), numeric_cols)
])
X = df[categorical_cols + numeric_cols]
y = df['estimate']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Definição de modelos e grids de hiperparâmetros
models_config = {
    'Linear Regression': {
        'model': LinearRegression(),
        'params': {}
    },
    'Ridge (L2)': {
        'model': Ridge(),
        'params': {'model__alpha': [0.1, 1.0, 10.0, 100.0]}
    },
    'Lasso (L1)': {
        'model': Lasso(),
        'params': {'model__alpha': [0.01, 0.1, 1.0]}
    },
    'Decision Tree': {
        'model': DecisionTreeRegressor(random_state=42),
        'params': {
            'model__max_depth': [5, 10, 20, None],
            'model__min_samples_split': [2, 5, 10]
        }
    },
    'Random Forest': {
        'model': RandomForestRegressor(random_state=42),
        'params': {
            'model__n_estimators': [100],
            'model__max_depth': [10, 20]
        }
    }
}

results = []
best_estimators = {}

for name, config in models_config.items():
    print(f"Treinando {name}...")
    pipeline = Pipeline(steps=[('preprocess', preprocessor), ('model', config['model'])])
    
    # Grid Search com Cross-Validation
    grid = GridSearchCV(pipeline, config['params'], cv=5, 
                        scoring='neg_mean_squared_error', 
                        return_train_score=True)
    grid.fit(X_train, y_train)
    
    # Melhores parâmetros e modelo
    best_model = grid.best_estimator_
    best_estimators[name] = best_model
    
    # Avaliação no conjunto de teste
    preds = best_model.predict(X_test)
    
    # Métricas
    results.append({
        'model': name,
        'best_params': grid.best_params_,
        'cv_rmse_mean': np.sqrt(-grid.best_score_),
        'train_rmse': np.sqrt(mean_squared_error(y_train, best_model.predict(X_train))),
        'test_rmse': mean_squared_error(y_test, preds, squared=False),
        'test_mae': mean_absolute_error(y_test, preds),
        'test_r2': r2_score(y_test, preds)
    })

results_df = pd.DataFrame(results)
display(results_df.sort_values('test_rmse'))

In [ ]:
# Análise de Overfitting: Variação da profundidade da árvore
depths = [1, 3, 5, 10, 15, 20, 30]
train_scores = []
test_scores = []

for d in depths:
    model = DecisionTreeRegressor(max_depth=d, random_state=42)
    pipe = Pipeline(steps=[('preprocess', preprocessor), ('model', model)])
    pipe.fit(X_train, y_train)
    
    train_scores.append(mean_squared_error(y_train, pipe.predict(X_train), squared=False))
    test_scores.append(mean_squared_error(y_test, pipe.predict(X_test), squared=False))

plt.figure(figsize=(10, 5))
plt.plot(depths, train_scores, marker='o', label='Treino RMSE')
plt.plot(depths, test_scores, marker='s', label='Teste RMSE')
plt.xlabel('Profundidade da Árvore (Max Depth)')
plt.ylabel('RMSE (menor é melhor)')
plt.title('Curva de Complexidade: Overfitting vs Underfitting')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
rf_model = models['Random Forest']
tree_steps = rf_model.named_steps['preprocess']
ohe = tree_steps.named_transformers_['cat']
encoded = ohe.get_feature_names_out(categorical_cols)
feature_names = np.concatenate([encoded, numeric_cols])
importances = rf_model.named_steps['model'].feature_importances_
top10 = np.argsort(importances)[::-1][:10]
plt.figure(figsize=(10, 5))
sns.barplot(x=importances[top10], y=feature_names[top10])
plt.title('Importância das features pelo Random Forest')
plt.tight_layout()
plt.show()

## Interpretação e Conclusão

### Descobertas Principais
- **Tendências:** A análise exploratória confirmou o aumento das taxas de overdose ao longo dos anos e diferenças significativas entre faixas etárias e tipos de drogas (painéis).
- **Modelagem:** O **Random Forest** e a **Decision Tree** (com profundidade ajustada) superaram os modelos lineares, indicando relações não-lineares complexas nos dados.
- **Regularização:** A aplicação de Ridge e Lasso ajudou a controlar a magnitude dos coeficientes nos modelos lineares, mas não foi suficiente para alcançar a performance dos modelos baseados em árvores.
- **Overfitting:** A curva de complexidade mostrou que árvores muito profundas (ex: depth > 15) tendem a overfitar (erro de treino cai, mas erro de teste estabiliza ou sobe). O GridSearch encontrou um ponto de equilíbrio.

### Limitações e Melhorias Futuras
- **Séries Temporais:** O modelo atual trata cada ano como uma observação independente. Modelos de séries temporais (ARIMA, Prophet) ou features de lag poderiam capturar melhor a dependência temporal.
- **Dados Externos:** Incluir dados socioeconômicos (desemprego, renda) poderia enriquecer a explicação das taxas.
- **Validação:** Uma validação cruzada baseada em tempo (Time Series Split) seria mais adequada para evitar vazamento de dados do futuro para o treino.